### Lesson 6: Question Answering


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.chat_models.cohere import ChatCohere
from langchain_community.embeddings.cohere import CohereEmbeddings
from langchain_community.vectorstores.chroma import Chroma

In [3]:
persist_directory = "./.chroma/"

In [4]:
embedding = CohereEmbeddings(model="embed-multilingual-light-v3.0")

In [12]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
)

In [13]:
vectordb._collection.count()

208

In [14]:
question = "What are major topics of the lectures given?"

docs = vectordb.max_marginal_relevance_search(query=question, k=3)

len(docs)

3

#### RetrievalQA Chain


In [15]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatCohere(model="command-light", temperature=0.1),
    retriever=vectordb.as_retriever(),
)

In [16]:
result = qa_chain.invoke({"query": question})

In [17]:
print(result["result"])

I do not have access to any information regarding lectures or university courses. If you like, you can give me some information about the topic you would like to know about, and I will do my best to provide you with a relevant response.


#### Prompt


In [18]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. \
    If you don't know the answer, just say that you don't know, \
    don't try to make up an answer. Use three sentences maximum. \
    Keep the answer as concise as possible. Always say "thanks for asking!" \
    at the end of the answer. \
{context}
Question: {question}
Helpful Answer:\n
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template=template)

In [19]:
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], template='Use the following pieces of context to answer the question at the end.     If you don\'t know the answer, just say that you don\'t know,     don\'t try to make up an answer. Use three sentences maximum.     Keep the answer as concise as possible. Always say "thanks for asking!"     at the end of the answer. {context}\nQuestion: {question}\nHelpful Answer:\n\n')

In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatCohere(model="command-light", temperature=0.1),
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [21]:
question = "Is probability a topic from the lectures given?"

In [24]:
result = qa_chain.invoke({"query": question})

In [25]:
print(result["result"])

From the provided information, it seems the lecture is about teaching probability and how it is intertwined with the Likelihood and parameters. Probability is a branch of mathematics that studies the likelihood or chance of a given event to occur. Using the provided information, the probability is a topic that is discussed and taught in the lecture. Hence, the answer is yes. 
Thanks for asking!


In [26]:
print(result["source_documents"][0].page_content)

theta as the probability of Y given X paramete rized by theta. And so this is going to be 
the product over my training set like that. Which is, in turn, going to be a product of those 
Gaussian densities that I wrot e down just now, right? Okay?  
Then in parts of notation, I guess, I define th is term here to be the likelihood of theta. 
And the likely of theta is just the probability of the data Y, right? Given X and prioritized 
by theta. To test the likeli hood and probability are often confused. So the likelihood of 
theta is the same thing as the probability of  the data you saw. So likely and probably are, 
sort of, the same thing. Except that when  I use the term likelihood I’m trying to 
emphasize that I’m taking this thing and view ing it as a function of  theta. Okay? So 
likelihood and for probability, th ey’re really the same thing except that when I want to 
view this thing as a function of theta holding X and Y fix are then called likelihood. 
Okay? So hopefully you hea

#### RetrievalQA chain types


In [27]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm=ChatCohere(model="command-light", temperature=0.1),
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce",
)

In [28]:
result = qa_chain_mr.invoke({"query": question})

In [29]:
print(result["result"])

I cannot give you a definitive answer to that question as it depends on the specific university course and syllabus. I can tell you that probability is a mathematical concept that relates to the likelihood of an event happening, or the measurement of how likely it is that a certain outcome will occur. 

Probability is a broad topic and is discussed in many university courses, typically within statistics and mathematics programs. It is widely used in many of the physical sciences, such as in the field of quantum mechanics where it is used to predict the likelihood of an electron occupying a specific energy state. It is also used in the biological sciences and sociology when recording and analyzing data. 

If you provide more information about the course you are interested in, I can give you a better idea of the topics discussed.


In [30]:
qa_chain_rf = RetrievalQA.from_chain_type(
    llm=ChatCohere(model="command-light", temperature=0.1),
    retriever=vectordb.as_retriever(),
    chain_type="refine",
)

result = qa_chain_rf.invoke({"query": question})

In [31]:
print(result["result"])

Thank you for providing this additional context from the lecture. Let's use this to refine our understanding. 

According to the lecture excerpt, Professor Andrew Ng makes two important points:
1. The choice of the sigmoid function in logistic regression - Professor Ng states that the sigmoid function, also known as the logistic function, is chosen primarily for two reasons: first, it belongs to the broader class of generalized linear models, which provides a useful conceptual framework; second, there are certain aesthetic considerations that make the choice particularly appealing in this context.
2. Probabilistic interpretation - Professor Ng assigns a probabilistic interpretation to the output of the sigmoid function, implying that the function estimates the probability of the binary outcome $Y$ taking the value of one.

Taking these points into account, the topic of probability is explicitly addressed in the lecture, particularly in the context of machine learning and statistical mo

#### RetrievalQA limitations

QA fails to preserve conversational history.


In [32]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatCohere(model="command-light", temperature=0.1),
    retriever=vectordb.as_retriever(),
)

In [33]:
question = "Is probability a topic of the lectures given?"

result = qa_chain.invoke({"query": question})

In [34]:
print(result["result"])

Most definitely! Probability is a fundamental concept that underlies many areas of computer science and data science, including statistics, machine learning, and data analysis. It is a topic that is frequently covered in lectures related to these fields. 

Probability enables us to quantify uncertainty, make informed decisions, and understand the likelihood of various events occurring. It plays a crucial role in understanding data, making predictions, and forming the basis for many machine learning algorithms. 

In lectures focused on statistics and data analysis, probability is typically introduced as a way to model uncertainty by assigning probabilities to events based on observed data. This allows for the analysis of patterns and the drawing of inferences from the data. 

In the context of machine learning, probability concepts are employed in algorithms that learn from data to make predictions or classify new, unseen instances. Techniques like Bayesian inference, probability theory

In [35]:
question = "why are those prerequesites needed?"

result = qa_chain.invoke({"query": question})

print(result["result"])

The specific prerequisites needed for a particular task or goal will depend on the nature of that task or goal. Generally, prerequisites are necessary to ensure that individuals have the knowledge, skills, and abilities needed to succeed in a particular role or endeavor. 

Educational prerequisites, for example, are often necessary for career paths where in-depth knowledge and understanding of a particular subject is required. Taking a physics degree, for instance, would require an understanding of advanced mathematical concepts and previous proficiency in physics and chemistry. 

Similarly, prerequisites may be needed in other aspects of life, such as learning a new skill or pursuing a hobby. For instance, learning how to play the piano would require prerequisite skills in reading music and having some knowledge of music theory. 

In some cases, prerequisites may not be necessary for a particular task or goal, especially if the objective is more straightforward or introductory in natu

Note, The LLM response varies. Some responses do include a reference to probability which might be gleaned from referenced documents. The point is simply that the model does not have access to past questions or answers, this will be covered in the next lesson.
